In [10]:
import requests
import time
import json
import time
import jwt
import os
import httpx

In [36]:
API_KEY = os.getenv('API_KEY')
BASE_URL = 'https://speech2text.ru/api/recognitions'

def transcribe_file(file_path, poll_interval=30, timeout=1000):
    """
    Отправляет аудиофайл на распознавание (2 спикера) и возвращает результат в формате JSON.
    
    :param file_path: путь к файлу
    :param poll_interval: интервал опроса статуса задачи (в секундах)
    :param timeout: максимальное время ожидания (в секундах)
    :return: dict (результат в формате JSON)
    """
    # 1. Отправка файла
    with open(file_path, 'rb') as f:
        files = {'file': f}
        data = {
            'lang': 'ru',
            'speakers': '2'  # указание числа спикеров
        }
        url = f'{BASE_URL}/task/file?api-key={API_KEY}'
        response = requests.post(url, files=files, data=data)

    if not response.ok:
        raise Exception(f'Ошибка при отправке файла: {response.text}')

    task_id = response.json().get('id')
    if not task_id:
        raise Exception('Не удалось получить task_id из ответа')

    print(f'Файл отправлен. TASK-ID: {task_id}')

    # 2. Ожидание завершения задачи
    status_url = f'{BASE_URL}/{task_id}?api-key={API_KEY}'
    start_time = time.time()

    while True:
        status_response = requests.get(status_url)
        if not response.ok:
            raise Exception(f'Ошибка при отправке файла: {response.text}')

        status_data = status_response.json()
        code = status_data.get('status', {}).get('code', 0)
        description = status_data.get('status', {}).get('description', 'Без описания')

        if code == 200:
            print(f'✅ Завершено: {description}')
            break
        elif code >= 500:
            raise Exception(f'❌ Ошибка сервера: {description}')
        else:
            print(f'⏳ Статус: {description}, жду {poll_interval} сек...')
            
        time.sleep(poll_interval)

    # 3. Получение результата
    result_url = f'{BASE_URL}/{task_id}/result/json?api-key={API_KEY}'
    result_response = requests.get(result_url)

    if not response.ok:
        raise Exception(f'Ошибка при отправке файла: {response.text}')

    return result_response.json()

In [ ]:
dialog1 = transcribe_file(file_path='out-89161357322-708-20201219-170417-1608386657.1552432.wav')
dialog2 = transcribe_file(file_path='out-89850561322-713-20200319-175108-1584629468.699634.wav')
dialog3 = transcribe_file(file_path='Минаков89636047195.wav')


In [ ]:
def format_dialogue(data):
    # Сопоставим ID → имя
    speaker_map = {s['id']: s['name'] for s in data.get('speakers', [])}
    
    lines = []
    for chunk in data.get('chunks', []):
        speaker_id = chunk.get('speaker')
        speaker_name = speaker_map.get(speaker_id, f'Спикер {speaker_id}')
        text = chunk.get('text', '').strip()
        if text:
            lines.append(f"{speaker_name}: {text}")

    return '\n'.join(lines)

dialog_result1 = format_dialogue(dialog1)
dialog_result2 = format_dialogue(dialog2)
dialog_result3 = format_dialogue(dialog3)

def save_dialog(text: str, name: str):
    filename = f"{name}.txt"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(text)
        
save_dialog(dialog_result1, 'dilog1')
save_dialog(dialog_result2, 'dilog2')
save_dialog(dialog_result3, 'dilog3')

In [37]:
service_account_id = os.getenv('service_account_id')
key_id = os.getenv('key_id')
private_key = os.getenv('private_key')
catalog_id = os.getenv('catalog_id')

In [12]:
now = int(time.time())
payload = {
        'aud': 'https://iam.api.cloud.yandex.net/iam/v1/tokens',
        'iss': service_account_id,
        'iat': now,
        'exp': now + 360}

# Формирование JWT
encoded_token = jwt.encode(
    payload,
    private_key,
    algorithm='PS256',
    headers={'kid': key_id})

url = 'https://iam.api.cloud.yandex.net/iam/v1/tokens'
x = requests.post(url,  headers={'Content-Type': 'application/json'}, json = {'jwt': encoded_token}).json()
token = x['iamToken']

### Подготовка данных

Пробуем разбить текст на параграфы

In [ ]:
import pandas as pd
from langchain_core.documents import Document
from docx import Document

def docx_to_dataframe(file_path):
    # Загружаем документ
    doc = Document(file_path)

    # Извлекаем непустые абзацы
    paragraphs = [p.text.strip() for p in doc.paragraphs if p.text.strip()]

    # Создаём DataFrame
    df = pd.DataFrame(paragraphs, columns=["paragraph"])

    return df

paragraphs_result = docx_to_dataframe('evgeniyjigliy.docx')

После просмотра содержимого датафрейма, результат не самый лучший в параграфах содержится совсем небольшее количество подходящей для нормального промпта информация.

Пробуем другой способ. Использование чанков по 8 параграфов со скользящим окном.

In [ ]:
import os

def sliding_window(paragraphs, window_size=8, step=4):
    chunks = []
    for i in range(0, len(paragraphs) - window_size + 1, step):
        chunk = paragraphs[i:i + window_size]
        chunks.append(chunk)
    return chunks

# Разбивка
chunked_paragraphs = sliding_window(paragraphs_result, window_size=8, step=4)

# Папка
os.makedirs("chunks", exist_ok=True)

for i, chunk_df in enumerate(chunked_paragraphs, start=1):
    # Извлекаем список строк из колонки 'paragraph'
    chunk = chunk_df["paragraph"].dropna().astype(str).tolist()
    
    # Объединяем в текст
    text = "\n\n".join(chunk)
    
    # Сохраняем в файл
    with open(f"chunks/chunk_{i}.txt", "w", encoding="utf-8") as f:
        f.write(text)

Результат тоже так себе чанки бьются на не логичные блоки с небольшим количеством значемой информации. Кажется для промпта использовать не самый лучший вариант.

In [16]:
from langchain_community.llms import YandexGPT
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.chains import HypotheticalDocumentEmbedder

Пробуем разбить книгу на логические блоки с помощью промпта. 

In [ ]:
# YandexGPT модель
model_version = "yandexgpt/latest"  # можно "rc" если нужно
model = YandexGPT(
    modelUri=f"gpt://{catalog_id}/{model_version}",
    iam_token=token,
    folder_id=catalog_id
)

# Промпт: структурировать текст по смысловым блокам
semantic_chunk_prompt = PromptTemplate.from_template("""
Ты — аналитик текста. Твоя задача — разбить представленный текст на логически завершённые смысловые блоки, сохранив оригинальный текст.

Каждый блок может состоять из одного или нескольких предложений, если они логически связаны. Не сокращай и не перефразируй текст — просто выдели смысловые фрагменты. 

Раздели их с помощью префиксов: "Блок 1:", "Блок 2:", и т.д.

Текст:
===
{chunk}
===
""")

# Цепочка: prompt → модель → парсер
chunk_to_blocks_chain = (
    {"chunk": RunnablePassthrough()}  # просто передаём текст
    | semantic_chunk_prompt
    | model
    | StrOutputParser()
)

# Чтение текста из файла
with open("chunks/chunk_15.txt", "r", encoding="utf-8") as f:
    chunk_text = f.read()

import os

# Папки
input_dir = "chunks"
output_dir = "result_block"
os.makedirs(output_dir, exist_ok=True)

# Получаем список файлов и сортируем по номеру
chunk_files = sorted(
    [f for f in os.listdir(input_dir) if f.startswith("chunk") and f.endswith(".txt")],
    key=lambda x: int("".join(filter(str.isdigit, x)))
)

# Обработка каждого чанка
for filename in chunk_files:
    file_path = os.path.join(input_dir, filename)
    
    # Чтение текста чанка
    with open(file_path, "r", encoding="utf-8") as f:
        chunk_text = f.read()

    try:
        # Обработка через цепочку
        result = chunk_to_blocks_chain.invoke(chunk_text)
    except Exception as e:
        print(f"Ошибка при обработке {filename}: {e}")
        continue

    # Сохранение результата
    block_num = "".join(filter(str.isdigit, filename))
    output_path = os.path.join(output_dir, f"block_{block_num}.txt")

    with open(output_path, "w", encoding="utf-8") as f:
        f.write(result)

    print(f"✅ Обработано: {filename} → {output_path}")

Блоков получилось много и каждый раз модель отрабатывала по разному, блоки не очень логически связанны и содержат не стуктурную информацию. Кажется надо остановиться на ручном разбиении текста на главы.

In [ ]:
import os

# Папка для файлов
folder = "chapsters"
os.makedirs(folder, exist_ok=True)

# Диапазон номеров
for i in range(12, 45):
    filename = f"chapster{i}.txt"
    filepath = os.path.join(folder, filename)
    with open(filepath, "w", encoding="utf-8") as f:
        pass  # Создаёт пустой файл

print(f"Создано {45 - 12} файлов в папке '{folder}'")

In [13]:
import os
import statistics
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import YandexGPTEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Путь к папке с главами
chapters_path = "chapsters"

# Загрузка .txt файлов как документы
documents = []
for filename in os.listdir(chapters_path):
    if filename.endswith(".txt"):
        loader = TextLoader(os.path.join(chapters_path, filename), encoding="utf-8")
        docs = loader.load()
        documents.extend(docs)

# Разбиение на чанки (если главы большие)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=300,
)
dataset = text_splitter.split_documents(documents)

# Статистика по длине
lengths = [len(doc.page_content) for doc in dataset]
m = statistics.mean(lengths)
sd = statistics.stdev(lengths)
print(f"Всего документов в базе: {len(dataset)}") 
print("Mean %d" % m)
print("SD %d" % sd)

# Векторизация
embeddings = YandexGPTEmbeddings(
    iam_token=token,
    folder_id=catalog_id,
    sleep_interval=0.1
)

# Создание или загрузка индекса
index_path = "travel_store"
if os.path.exists(index_path):
    storage = FAISS.load_local(index_path, embeddings, allow_dangerous_deserialization=True)
    print("✅ Индекс загружен из диска.")
else:
    storage = FAISS.from_documents(dataset, embedding=embeddings)
    storage.save_local(index_path)
    print("✅ Индекс создан и сохранён.")

Всего документов в базе: 120
Mean 1919
SD 1059
✅ Индекс создан и сохранён.


Проверка работоспособности и логичности ответа:

In [14]:
results = storage.similarity_search("как правильно работать с возражениями?", k=3)
for i, res in enumerate(results, 1):
    print(f"\n--- Результат {i} ---\n{res.page_content}")


--- Результат 1 ---
настоящая цель работы с возражениями: чтобы человек самостоятельно изменил свою точку зрения при помощи новой информации.
Его мнение субъективное, и оно может быть изменено им же самим при наличии новой информации, что и представляет собой ваш аргумент, который нужно готовить заранее! Самое сложное возражение — то, на которое у нас нет убедительного, заранее заготовленного аргумента!
Вообще возражение — это хорошо. Самый «тяжелый» кли- ент — тот, который молчит и кладет трубку! Возражая, человек заявляет о своей точке зрения, а с этим уже можно работать. Если клиент возражает и при этом не уходит, значит, ему все еще от нас что-то нужно!
Если взять на себя ответственность, то чаще всего источником возражений является отсутствие разведки: когда мы начинаем предлагать, не поняв запросов человека и выгод, наши мнения часто не совпадают, это вызывает отрицательную реакцию и воз- ражения. Клиент не задумывается о нашей ответственности , у него есть семь основных причин

In [27]:
gpt = YandexGPT(iam_token=token, folder_id=catalog_id)

def extract_growth_points_and_question(call_text: str, system_prompt: str) -> tuple[str, str]:
    prompt = f"""{system_prompt}

Вот расшифровка звонка:

{call_text}

Ответь в формате:
ТОЧКИ РОСТА:
...

ЗАПРОС ДЛЯ ПОИСКА В КНИГЕ:
..."""

    # LangChain LLM-совместимый вызов
    response = gpt.generate(prompts=[prompt])

    # Получаем текст из LLMResult
    response_text = response.generations[0][0].text

    # Простая парсилка
    growth = response_text.split("ЗАПРОС ДЛЯ ПОИСКА В КНИГЕ:")[0].replace("ТОЧКИ РОСТА:", "").strip()
    question = response_text.split("ЗАПРОС ДЛЯ ПОИСКА В КНИГЕ:")[1].strip()
    return growth, question

In [29]:
def generate_coaching(call_text: str, context_docs: list[str]) -> str:
    context = "\n\n".join([doc.page_content for doc in context_docs])

    prompt = f"""
Вот расшифровка звонка сотрудника с клиентом:

{call_text}

Ниже представлены отрывки из книги по продажам, которые помогут сотруднику стать лучше:

{context}

На основе звонка и главы из книги, напиши:
1. Похвалу (что хорошо),
2. Зоны роста (что улучшить),
3. Конкретные рекомендации с опорой на книгу.
"""

        # Передаём список строк в prompts
    response = gpt.generate(prompts=[prompt])

    # Извлекаем результат из LLMResult
    return response.generations[0][0].text

In [21]:
system_prompt = """Ты — коуч по продажам. Твоя задача — анализировать разговоры сотрудников с клиентами и находить точки роста.

1. Прочитай текст звонка. Выдели, что сотрудник мог бы улучшить: фразы, поведение, логика, упущенные возможности.
2. Сформулируй **2–3 чёткие точки роста** — кратко, по делу.
3. Затем **сформулируй один вопрос**, который можно задать векторной базе знаний (главам из книги по продажам), чтобы найти подходящие советы или техники."""


In [30]:
call_text = open("dilog1.txt", "r", encoding="utf-8").read()

growth, question = extract_growth_points_and_question(call_text, system_prompt)
print("🟡 Точки роста:\n", growth)
print("🔵 Запрос к базе знаний:\n", question)

results = storage.similarity_search(question, k=3)
coaching = generate_coaching(call_text, results)
print("\n🟢 Коучинг:\n", coaching)

🟡 Точки роста:
 1. Сотрудник не задал уточняющих вопросов о потребностях клиента, не выяснил, какие именно сорта кофе клиент предпочитает, какие дополнительные аксессуары или услуги могут быть интересны.
2. Сотрудник не предложил более подробно рассказать о преимуществах подменного фонда и возможности обмена кофемашины в случае поломки.
3. Сотрудник не акцентировал внимание на дополнительных скидках и акциях, которые могли бы заинтересовать клиента.
🔵 Запрос к базе знаний:
 «Как выявить потребности клиента и предложить ему наиболее подходящие решения?»

🟢 Коучинг:
 **Похвала:**

Сотрудник продемонстрировал умение вести диалог с клиентом, задавать правильные вопросы и предлагать подходящие решения. Он смог установить контакт с клиентом и выяснить его потребности, что является важным навыком для успешного продавца.

**Зоны роста:**

1. Необходимо более активно использовать универсальные основные вопросы, чтобы получить больше информации о потребностях клиента.
2. Следует уделять больше в

In [32]:
call_text = open("dilog2.txt", "r", encoding="utf-8").read()

growth, question = extract_growth_points_and_question(call_text, system_prompt)
print("🟡 Точки роста:\n", growth)
print("🔵 Запрос к базе знаний:\n", question)

results = storage.similarity_search(question, k=3)
coaching = generate_coaching(call_text, results)
print("\n🟢 Коучинг:\n", coaching)

🟡 Точки роста:
 1. Необходимо было более чётко и структурированно изложить информацию о предложении, чтобы собеседник лучше понял суть и преимущества предложения.
2. Не было задано уточняющих вопросов о потребностях собеседника, что могло бы помочь подобрать более персонализированное предложение.
🔵 Запрос к базе знаний:
 Какие техники задавания вопросов и выявления потребностей клиента можно использовать в продажах, чтобы сделать презентацию более персонализированной и эффективной?

🟢 Коучинг:
 **Похвала:**

Сотрудник успешно провёл разговор с клиентом, проявил умение слушать и задавать уточняющие вопросы, что позволило ему лучше понять потребности клиента и предложить подходящее решение.

**Зоны роста:**

1. Необходимо более активно использовать алгоритмы и скрипты для ведения разговора, чтобы обеспечить более структурированный и целенаправленный подход.
2. Следует уделять больше внимания сбору и анализу вопросов клиентов, чтобы заранее готовить на них исчерпывающие ответы.
3. Важно р

In [33]:
call_text = open("dilog3.txt", "r", encoding="utf-8").read()

growth, question = extract_growth_points_and_question(call_text, system_prompt)
print("🟡 Точки роста:\n", growth)
print("🔵 Запрос к базе знаний:\n", question)

results = storage.similarity_search(question, k=3)
coaching = generate_coaching(call_text, results)
print("\n🟢 Коучинг:\n", coaching)

🟡 Точки роста:
 1. Необходимо было более активно предлагать новые модели капучинаторов, подчёркивая их надёжность и преимущества.
2. Следует уделить больше внимания выявлению потребностей клиента и предложить наиболее подходящий вариант решения проблемы.
🔵 Запрос к базе знаний:
 Как эффективно выявлять потребности клиента и предлагать решения, которые наилучшим образом соответствуют их ожиданиям?

🟢 Коучинг:
 **Похвала:**

Сотрудник продемонстрировал умение перехватывать инициативу в разговоре, задавать уточняющие вопросы и предлагать решение, соответствующее потребностям клиента.

**Зоны роста:**

1. Необходимо более активно использовать универсальные основные вопросы, чтобы лучше понимать потребности клиента и предлагать более персонализированные решения.
2. Следует уделять больше внимания сопровождению клиента после совершения покупки, чтобы укрепить его лояльность и стимулировать повторные обращения.
3. Важно работать над умением обходить вопросы о цене, чтобы не создавать у клиент